# **Titanic 데이터를 이용한 <span style="color:darkgreen">머신러닝</span> 문제**
---

> **<span style="color:red">다음 문항을 풀기 전에 </span>아래 코드를 실행하시오.**<br>
> 반드시 코드와 주석을 읽고 문제를 푸시오. <br>
> 반드시 출력된 데이터 설명을 읽고 문제를 푸시오.

---



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml

x, y = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)

print( x.shape, y.shape)
print(y[:4])
x.head()

# **Q1. x에서 다음의 컬럼을 삭제하시오.**
---------------------------------
**삭제할 컬럼들**
1. name
2. ticket
2. boat
3. body
4. home.dest
---------------------------

In [ ]:
####################
## Your code here ##
####################

x.drop(['name','ticket', 'boat', 'body', 'home.dest'], axis=1, inplace=True)
x.head(3)

# Q2. x의 cabin 컬럼을 다음 조건에 맞추어 변형하시오.
-------------------
* 값의 맨 앞글자만 남긴다.
    * 예시) C22 C26 -> C
    * 예시) B5 -> B
* 값이 빈값일 경우, 'N'을 남긴다.
--------------------

In [ ]:
####################
## Your code here ##
####################
x.loc[ x['cabin'].isna(), 'cabin'] = 'N'
temp = [ cabin[0] for cabin in x['cabin'] ]
x['cabin'] = temp

x.head(3)

# Q3. 다음 조건에 맞추어 x에 새로운 컬럼 fs를 만드시오.
-------------------
* sibsp와 parch의 합을 기준으로
    - 1이하 : 'a' 를 값으로 갖는다.
    - 1초과 4이하 : 'b' 를 값으로 갖는다.
    - 4초과 : 'c' 를 값으로 갖는다.
----------------

In [ ]:
####################
## Your code here ##
####################
temp = x['sibsp'] + x['parch']
temp = pd.cut(temp, bins=[-1, 1, 4, np.inf], labels=['a','b','c'])
x['fs'] = temp

x.head(3)

# Q4. x와 y를 트레이닝/벨리데이션/테스트 셋으로 분리하시오.
----------------------
* 변수명 규칙 :
    - x_train, x_valid, x_test
    - y_train, y_valid, y_test
* train : valid : test = 6 : 2 : 2
* random state, seed 등은 2021로 고정
------------------------

In [ ]:
####################
## Your code here ##
####################
from sklearn.model_selection import train_test_split as tts

x_train, x_test, y_train, y_test = tts(x, y, test_size=0.2, random_state=2021)
x_train, x_valid, y_train, y_valid = tts(x_train, y_train, test_size=2/8, random_state = 2021)

# Q5. 카테고리 데이터들에 빈값이 있다면, 최빈값을 찾아 imputing하시오.
----------------------------
* 모든 전처리 규칙은 트레이닝 셋을 바탕으로 찾는다.
* 대상 컬럼 : pclass, cabin, embarked
* 모든 x들에 대해 진행
----------------------------

In [ ]:
####################
## Your code here ##
####################
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='most_frequent')

x_train[['pclass', 'cabin', 'embarked']] = imputer.fit_transform(x_train[['pclass', 'cabin', 'embarked']])
x_valid[['pclass', 'cabin', 'embarked']] = imputer.transform( x_valid[['pclass', 'cabin', 'embarked']] )
x_test[['pclass', 'cabin', 'embarked']] = imputer.transform( x_test[['pclass', 'cabin', 'embarked']] )

# Q6. 연속형 데이터들에 빈값이 있다면, 중앙값을 찾아 imputing하시오.
----------------------------
* 모든 전처리 규칙은 트레이닝 셋을 바탕으로 찾는다.
* 대상 컬럼 : age, fare
* 모든 x들에 대해 진행
----------------------------

In [ ]:
####################
## Your code here ##
####################
from sklearn.impute import SimpleImputer

imputer2 = SimpleImputer(strategy='median')

x_train[['age', 'fare']] = imputer2.fit_transform(x_train[['age', 'fare']])
x_valid[['age', 'fare']] = imputer2.transform( x_valid[['age', 'fare']] )
x_test[['age', 'fare']] = imputer2.transform( x_test[['age', 'fare']] )

# Q7. 카테고리 데이터들을 dummy 변수화 하시오.
-------------------------------
* 모든 전처리 규칙은 트레이닝셋을 바탕으로 찾는다.
* 대상 컬럼 : pclass, cabin, embarked, fs, sex
* 더미변수를 만든 후, 원본 변수는 삭제한다.
* 모든 x들에 대해 진행
* 카테고리 안에 4개 클래스가 있다면, 3개 컬럼이 만들어져야 한다.
* x_valid와 x_test의 컬럼들은 x_train의 컬럼과 종류와 순서가 같아야 한다.
-------------------------------

In [ ]:
####################
## Your code here ##
####################
x_train = pd.get_dummies(x_train, columns=['pclass', 'cabin', 'embarked', 'fs', 'sex'], drop_first= True)
x_valid = pd.get_dummies(x_valid, columns=['pclass', 'cabin', 'embarked', 'fs', 'sex'], drop_first= True)
x_test = pd.get_dummies(x_test, columns=['pclass', 'cabin', 'embarked', 'fs', 'sex'], drop_first= True)

x_valid = x_valid[x_train.columns]
x_test = x_test[x_train.columns]

In [ ]:
x_train.head()

# Q8. 다음 조건에 맞추어 의사결정나무들을 학습시키시오.
-----------------------
* max depth는 3으로 고정한다.
* min samples leaf 1개짜리 나무부터 1개씩 늘려서 min samples leaf가 30개인 나무까지, 총 30개의 의사결정 나무를 학습시킨다.
* 학습된 의사결정나무들은 trees 변수에 리스트로 담아둔다.
-----------------------

In [ ]:
####################
## Your code here ##
####################

from sklearn.tree import DecisionTreeClassifier

trees = []

for i in range(1, 31):
    dtr = DecisionTreeClassifier(max_depth=3, min_samples_leaf=i)
    dtr.fit(x_train, y_train)
    trees.append(dtr)

# Q9. 학습된 의사결정나무들의 성능을 벨리데이션셋에서 시각화 하시오.
-------------------
* y가 1 (생존)일 때를 positive, 0 (사망)일 때를 negative로 둔다.
* 벨리데이션 셋에서 sensitivity를 계산하여 시각화 한다.
* x축은 min sample leaf의 수, y축은 sensitivity
* line chart를 그린다.
* grid line을 추가한다.
--------------------

In [ ]:
####################
## Your code here ##
####################

sensitivities = []

for i in range(1, 31):
    dtr = trees[i-1]
    y_pred = dtr.predict(x_valid)

    denominator = y_valid.astype(int).sum()
    numerator = y_pred[y_valid == "1"].astype(int).sum()

    sensitivities.append( numerator/denominator   )

plt.figure(figsize=(10,6))
plt.plot(range(1,31), sensitivities)
plt.grid()
plt.show()

# Q10. Q9를 참고하여 가장 성능 좋은 (동일 성능이라면 가장 min sample leaf가 큰) 나무를 골라 테스트셋 위에서 평가하시오.
----------------
1. accuracy를 주석으로 명시하시오.
2. 1을 positive로 두었을 때, sensitivity를 주석으로 명시하시오.
3. 1을 positive로 두었을 때, specificity를 주석으로 명시하시오.

In [ ]:
####################
## Your code here ##
####################

from sklearn.metrics import classification_report

best_tree = trees[24]
y_pred = best_tree.predict(x_test)

print(classification_report(y_test, y_pred, target_names=['negative', 'positive']))

# accuracy : 83%
# sensitivity : 81% 
# specificity : 84%

'''
Note that in binary classification, 
recall of the positive class is also known as “sensitivity”; 
recall of the negative class is “specificity”.
'''